<a href="https://colab.research.google.com/github/SusimRoy/Real-Time-Mask-Detection/blob/main/Mask_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 97% 320M/330M [00:03<00:00, 83.7MB/s]
100% 330M/330M [00:03<00:00, 107MB/s] 


In [ ]:
!unzip face-mask-12k-images-dataset.zip

In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
dir_type = ["Train","Test","Validation"]
cat = ["WithMask","WithoutMask"]

training_data = []
test_data = []
val_data=[]

IMG_SIZE = 150

path = "Face Mask Dataset/"

def create_data():
    for item in dir_type:
        data_path = path+item
        X = []
        y = []
        for folder in os.listdir(data_path):
            if cat.index(folder) == 0:
                labels = [1.0, 0.0]
            else:
                labels = [0.0, 1.0]
            for image in os.listdir(data_path+"/"+folder):
                img = cv2.imread(data_path+"/"+folder+"/"+image)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (125, 125))
                X.append(img)
                y.append(labels)
            if item == "Train":
                training_data.append((X, y))
            if item=="Test":
                test_data.append((X, y))
            if item=="Validation":
                val_data.append((X, y))

create_data()

In [ ]:
train_images = np.array(training_data[0][0])/255.0
train_images = train_images.reshape(-1, 125, 125,3)
train_labels = np.array(training_data[0][1])

test_images = np.array(test_data[0][0])/255.0
test_images = test_images.reshape(-1, 125, 125, 3)
test_labels = np.array(test_data[0][1])

val_images = np.array(val_data[0][0])/255.0
val_images = val_images.reshape(-1, 125, 125, 3)
val_labels = np.array(val_data[0][1])
print(train_images.shape)
print(test_images.shape)
print(val_images.shape)

(10000, 125, 125, 3)
(992, 125, 125, 3)
(800, 125, 125, 3)


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten
from tensorflow.keras.applications import VGG19


data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
        keras.layers.experimental.preprocessing.RandomZoom((0.2, 0.3)),
    ]
)
# model = keras.models.Sequential()
# model.add(Conv2D(64, (5, 5), input_shape=(None, 100, 100, 1)))
# model.add(Conv2D(64, (5, 5)))

# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(2))
# model.add(Activation('softmax'))

# model.build(input_shape=(None, 100, 100, 1))
# model.summary()


base_model = tf.keras.applications.VGG19(input_shape=(125, 125, 3), include_top=False, weights='imagenet')
for layer in base_model.layers[2:]:
    layer.trainable = False
# base_model.trainable =False
flattening = tf.keras.layers.Flatten()
pre_layer1 = tf.keras.layers.Dense(256,activation="relu")
pre_layer2 = tf.keras.layers.Dense(64, activation="relu")
prediction_layer = tf.keras.layers.Dense(2, activation="softmax")

inputs = tf.keras.Input(shape=(125, 125, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True)
# x = tf.keras.layers.Dropout(0.2)(x)
x = flattening(x)
x = pre_layer1(x)
x=pre_layer2(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 125, 125, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 125, 125, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130   

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 10
model.fit(train_images, train_labels, batch_size=32, epochs=epochs, validation_data=(val_images, val_labels), shuffle=True)

Epoch 1/10
313/313 [==============================] - 70s 113ms/step - loss: 0.0756 - accuracy: 0.9705 - val_loss: 0.0067 - val_accuracy: 0.9987
Epoch 2/10
313/313 [==============================] - 35s 110ms/step - loss: 0.0369 - accuracy: 0.9876 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 3/10
313/313 [==============================] - 36s 114ms/step - loss: 0.0291 - accuracy: 0.9896 - val_loss: 0.0051 - val_accuracy: 0.9987
Epoch 4/10
313/313 [==============================] - 38s 121ms/step - loss: 0.0216 - accuracy: 0.9918 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 5/10
313/313 [==============================] - 38s 121ms/step - loss: 0.0229 - accuracy: 0.9919 - val_loss: 0.0052 - val_accuracy: 0.9987
Epoch 6/10
313/313 [==============================] - 39s 124ms/step - loss: 0.0217 - accuracy: 0.9924 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 7/10
313/313 [==============================] - 39s 125ms/step - loss: 0.0169 - accuracy: 0.9936 - val_loss: 0.0071 - val_ac

In [ ]:
# score the model
score = model.evaluate(test_images, test_labels)
print("loss: ", score[0])
print("accuracy : ", score[1]*100, "%")

31/31 [==============================] - 2s 53ms/step - loss: 0.0114 - accuracy: 0.9990
loss:  0.01138278003782034
accuracy :  99.89919066429138 %


In [ ]:
model.save("Mask_detect.h5")
model.save("./")

INFO:tensorflow:Assets written to: ./assets


In [ ]:
new_model = tf.keras.models.load_model('Mask_detect.h5')

new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 125, 125, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 125, 125, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130   